In [ ]:
import os
import shutil
import pandas as pd
import geopandas as gpd
import rasterio
import s2cloudless
import numpy as np
import rasterio.warp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import mysecrets
import fetch_from_cluster
import modify_images
import rsutils.utils
import create_stack
import stack_ops

In [ ]:
zip_filepath = fetch_from_cluster.download_file_from_cluster(
    sshcreds = mysecrets.SSH_UMD_SASIRAJANN,
    # remotepath = '/gpfs/data1/cmongp2/sasirajann/data/misc/165bca4_s2l1c.zip',
    remotepath = '/gpfs/data1/cmongp2/sasirajann/fetch_satdata/data/gee_compare/grid25.zip',
    # download_folderpath = '../data/cluster_files',
    download_filepath = '../data/cluster_files/grid25_v2.zip'
    # overwrite = True,
)

In [ ]:
stack_folderpath = '../data/gee_compare_grid25_v2'
# stack_folderpath = '../data/165bca4_s2l1c_stack'

In [ ]:
if not os.path.exists(stack_folderpath):
    shutil.unpack_archive(zip_filepath, stack_folderpath)

In [ ]:
bands, metadata = create_stack.load_stack(stack_folderpath)

In [ ]:
bands.shape, metadata['bands'], metadata['geotiff_metadata']

In [ ]:
startdate = datetime.datetime(2021, 3, 1)
enddate = datetime.datetime(2021, 5, 1)

In [ ]:
cmk = bands[:,:,:,-1]
bands_wo_cmk = bands[:,:,:,:-1]

cmk.shape, bands_wo_cmk.shape

In [ ]:
metadata['bands'].remove('CMK')

In [ ]:
cloud_threshold = 0.65 * 10000

In [ ]:
bands_wo_cmk[np.where(cmk >= cloud_threshold)] = 0

In [ ]:
bands_wo_cmk.shape

In [ ]:
(bands_wo_cmk == 0).sum() / np.prod(bands_wo_cmk.shape)

In [ ]:
mosaiced_bands, mosaiced_metadata \
= stack_ops.median_mosaic(
    bands = bands_wo_cmk, 
    metadata = metadata,
    startdate = datetime.datetime(2021, 3, 1),
    enddate = datetime.datetime(2021, 5, 1),
    mosaic_days = 20,
)

In [ ]:
mosaiced_bands.shape

In [ ]:
mosaiced_bands[0, :, :, 0]

In [ ]:
mosaiced_metadata